In [1]:
# Librerias
import pandas as pd # Guardar informacion en dataframes
import glob # Buscar un patron de un archivo
import os

In [25]:
# Cargar el archivo CSV con manejo de errores
def cargar_csv(ruta):
    try:
        df = pd.read_csv(ruta)
        return df
    except FileNotFoundError:
        print(f"Error: El archivo {ruta} no se encontró.")
        return None
    except pd.errors.ParserError:
        print("Error: Hubo un problema al analizar el archivo CSV.")
        return None

# Guardar genes agrupados por pathways en un solo archivo
def guardar_genes_por_pathway(df, salida):
    if df is not None:
        df_filtrado = df[['description', 'category', 'term', 'inputGenes', 'preferredNames']]
        try:
            # Leer el archivo existente solo una vez al principio
            df_existente = pd.read_csv(salida)
            # Eliminar duplicados antes de agregar nuevas filas
            df_completo = pd.concat([df_existente, df_filtrado], ignore_index=True).drop_duplicates(subset=['description', 'category', 'term', 'inputGenes', 'preferredNames'])
        except FileNotFoundError:
            # Si no existe, crear uno nuevo con la información filtrada
            df_completo = df_filtrado
        
        df_completo.to_csv(salida, index=False)
    else:
        print("No se pudo guardar el archivo porque hubo un error al cargar los datos.")

# Cargar y procesar los archivos CSV
rutas_csv = [
    "ShinyGo/Vector/STRING_enrichmentInterPro.csv",
    "ShinyGo/Vector/STRING_enrichmentKEGG.csv"
]

salida_csv = "ShinyGo/Vector/genes_pathway.csv"  # Archivo de salida único

# Procesar ambos archivos
for ruta in rutas_csv:
    df = cargar_csv(ruta)
    guardar_genes_por_pathway(df, salida_csv)

In [22]:
# Cargar el archivo CSV
df = pd.read_csv("ShinyGo/Vector/genes_pathway.csv")

# Separar los genes en filas y guardar en archivos TSV por pathway
for index, row in df.iterrows():
    input_genes = row['inputGenes'].split(',')
    preferred_names = row['preferredNames'].split(',')
    
    # Crear un DataFrame con las columnas inputGenes y preferredNames
    df_genes = pd.DataFrame({
        'inputGenes': input_genes,
        'preferredNames': preferred_names
    })
    
    # Crear el nombre del archivo basado en la descripción de la ruta (pathway)
    pathway_name = row['description'].replace(" ", "_")  # Reemplazar espacios por guiones bajos
    pathway_name = pathway_name.replace("/", "_")  # Reemplazar las barras "/" por guiones bajos
    pathway_name = pathway_name.replace("\\", "_")  # Reemplazar las barras invertidas "\\" por guiones bajos
    pathway_name = pathway_name.replace(":", "_")  # Reemplazar los dos puntos ":" por guiones bajos
    pathway_name = pathway_name.replace("-", "_")  # Reemplazar los dos puntos "-" por guiones bajos
    
    archivo_salida = f"ShinyGo/Vector/pathway/{pathway_name}.tsv"
    
    # Guardar el DataFrame en un archivo TSV
    df_genes.to_csv(archivo_salida, sep='\t', index=False)

In [18]:
# Cargar el archivo CSV
df = pd.read_csv("ShinyGo/Vector/geneInfo.csv")

# Seleccionar las columnas relevantes
df_selected = df[["User_input", "STRINGdb ID"]]

# Guardar en un archivo .tsv
df_selected.to_csv("ShinyGo/Vector/geneInfo.tsv", sep='\t', index=False)

In [19]:
# Cargar el archivo geneInfo.tsv
gene_info = pd.read_csv("ShinyGo/Vector/geneInfo.tsv", sep='\t')

# Lista de rutas de importancia (basadas en su nombre en los archivos)
important_pathways = [
    "Glycolysis__Gluconeogenesis",  # Glycolysis / Gluconeogenesis
    "Neuroactive_ligand-receptor_interaction",  # Neuroactive ligand-receptor interaction
    "Oxidative_phosphorylation",  # Oxidative phosphorylation
    "P_loop_containing_nucleoside_triphosphate_hydrolase",  # P-loop containing nucleoside triphosphate hydrolase
    "Protein_processing_in_endoplasmic_reticulum"  # Protein processing in endoplasmic reticulum
    "Spliceosome",  # Spliceosome
    "Ubiquitin_mediated_proteolysis",  # Ubiquitin mediated proteolysis
]

# Usar glob para obtener todos los archivos .tsv en la carpeta de vías metabólicas
pathways_files = glob.glob("ShinyGo/Vector/pathway/*.tsv")

# Crear un DataFrame vacío para almacenar las filas que coinciden
matching_rows = pd.DataFrame()

# Comparar STRINGdb ID con preferredNames en cada archivo de vías
for file in pathways_files:
    # Verificar si el nombre del archivo contiene una de las rutas importantes
    if any(pathway in file for pathway in important_pathways):
        # Cargar el archivo de la vía metabólica
        pathway_data = pd.read_csv(file, sep='\t')
        
        # Verificar si hay coincidencias entre STRINGdb ID y preferredNames
        merged_data = pd.merge(gene_info[['User_input', 'STRINGdb ID']], pathway_data[['preferredNames']], 
                               left_on='STRINGdb ID', right_on='preferredNames', how='inner')
        
        # Si hay coincidencias, agregar las columnas necesarias a matching_rows
        if not merged_data.empty:
            matching_rows = pd.concat([matching_rows, merged_data[['User_input']]])

# Eliminar duplicados de matching_rows
matching_rows.drop_duplicates(inplace=True)

# Renombrar la columna 'User_input' a 'Protein_ID'
matching_rows = matching_rows.rename(columns={'User_input': 'Protein_ID'})

# Guardar el archivo final
matching_rows.to_csv("ShinyGo/VectorBase_Rprolixus.tsv", sep='\t', index=False)

In [20]:
# Leer el archivo .tsv
tsv_file = "ShinyGo/VectorBase_Rprolixus.tsv"
df_tsv = pd.read_csv(tsv_file, sep='\t')

# Extraer la primera columna, convertirla a un set para eliminar duplicados
protein_set = set(df_tsv.iloc[:, 0])

# Guardar el resultado sin nombre de columna
protein_df = pd.DataFrame(list(protein_set))
protein_df.to_csv("ShinyGo/VectorBase_Rprolixus_SinProteinID.tsv", sep='\t', index=False, header=False)